In [2]:
from sys import exit, exc_info, argv
import numpy as np
import pandas as pd

!pip3 install git+https://github.com/slremy/netsapi --user --upgrade

from netsapi.challenge import *
from netsapi.visualisation import *


  Cloning https://github.com/slremy/netsapi to /private/var/folders/qj/6zzv4hhx5fzbkm01x5qr69800000gn/T/pip-req-build-8827peqc
  Running command git clone -q https://github.com/slremy/netsapi /private/var/folders/qj/6zzv4hhx5fzbkm01x5qr69800000gn/T/pip-req-build-8827peqc
  Stored in directory: /private/var/folders/qj/6zzv4hhx5fzbkm01x5qr69800000gn/T/pip-ephem-wheel-cache-dxqfqhs6/wheels/9e/73/c9/86a9cc2460e11b3ce5b0a5ebd2d9d332a68afe0941659967fa
Successfully built netsapi
  Found existing installation: netsapi 1.1
    Uninstalling netsapi-1.1:
      Successfully uninstalled netsapi-1.1


In [85]:
from collections import defaultdict

env = ChallengeSeqDecEnvironment()

Q = defaultdict(lambda : 0.) # Q-function
n = defaultdict(lambda : 1.) # number of visits

def actionSpace(resolution):
    x,y = np.meshgrid(np.arange(0,1+resolution,resolution), np.arange(0,1+resolution,resolution))
    xy = np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)
    return xy.round(2).tolist()

#HyperParameters
epsilon = 0.9
gamma = 0.8
action_resolution = 0.5
episode_number = 20 #for submission this is fixed as 20

learning_rate = 1

#Set-up
actions = actionSpace(action_resolution)
actionspace = range(len(actions)-1)
greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])
count = 0

policies = []
rewards = []
#Training of Q Table

print(Q)

for _ in range(episode_number):
    env.reset()
    nextstate = env.state
    while True:
        state = nextstate
        # Epsilon-Greedy
        if epsilon > random.random() :
            action = random.choice(actionspace)
            print('random_action',action)
        else :
            action = greedy_action(state)

        env_action = actions[action] #convert to ITN/IRS
        print('env_action', env_action)
        nextstate, reward, done, _ = env.evaluateAction(env_action)
    
        epsilon = epsilon * 0.8
        # Q-learning
        """
        if done :
            Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward - Q[(state,action)] )
            break
        else :
            Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )
        """
        if done :
            Q[(state,action)] = Q[(state,action)] + learning_rate * ( reward - Q[(state,action)] )
            break
        else :
            Q[(state,action)] = Q[(state,action)] + learning_rate * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )
        
        learning_rate = learning_rate * 0.999
        
#Greedy Policy Learnt from Q Table
best_policy = {state: list(actions[greedy_action(state-1)]) for state in range(1,6)}
best_reward = env.evaluatePolicy(best_policy)
print(best_policy, best_reward)

defaultdict(<function <lambda> at 0x13edae2f0>, {})
random_action 3
env_action [0.0, 0.5]
105  Evaluations Remaining
env_action [0.0, 0.0]
104  Evaluations Remaining
random_action 3
env_action [0.0, 0.5]
103  Evaluations Remaining
random_action 4
env_action [0.5, 0.5]
102  Evaluations Remaining
env_action [0.0, 0.0]
101  Evaluations Remaining
env_action [0.0, 0.5]
100  Evaluations Remaining
random_action 2
env_action [1.0, 0.0]
99  Evaluations Remaining
env_action [0.0, 0.5]
98  Evaluations Remaining
env_action [0.5, 0.5]
97  Evaluations Remaining
env_action [0.0, 0.0]
96  Evaluations Remaining
env_action [0.0, 0.5]
95  Evaluations Remaining
env_action [1.0, 0.0]
94  Evaluations Remaining
env_action [0.0, 0.5]
93  Evaluations Remaining
random_action 0
env_action [0.0, 0.0]
92  Evaluations Remaining
env_action [0.0, 0.0]
91  Evaluations Remaining
env_action [0.0, 0.5]
90  Evaluations Remaining
env_action [1.0, 0.0]
89  Evaluations Remaining
env_action [0.0, 0.5]
88  Evaluations Remainin

In [86]:
Q

defaultdict(<function __main__.<lambda>()>,
            {(1, 3): 117.45582055305213,
             (2, 0): 0.1670761712022857,
             (2, 1): 0.0,
             (2, 2): 111.18104183801076,
             (2, 3): 0.0,
             (2, 4): 0.0,
             (2, 5): 0.0,
             (2, 6): 0.0,
             (2, 7): 0.0,
             (3, 0): 0.0,
             (3, 1): 0.0,
             (3, 2): 0.0,
             (3, 3): 31.93006591399085,
             (3, 4): 0.0,
             (3, 5): 0.0,
             (3, 6): 0.0,
             (3, 7): 0.0,
             (4, 0): 0.3028866938562811,
             (4, 1): 0.0,
             (4, 2): 0.0,
             (4, 3): 0.0,
             (4, 4): 7.338069129121742,
             (4, 5): 0.0,
             (4, 6): 0.0,
             (4, 7): 0.0,
             (5, 0): -0.18128846236414115,
             (5, 1): 1.487735438507334,
             (5, 2): 0.0,
             (5, 3): 0.0,
             (5, 4): 0.0,
             (5, 5): 0.0,
             (5, 6): 0.0,
     

* action space [0, 0.5, 1] : 
    * a = 1
        * epsilon = 0.9 => 96.58
        * epsilon = 0.9 * (0.99)^loop => 76.43
        * epsilon = 0.9 * (0.9)^loop => 177.65
        * epsilon = 0.9 * (0.8)^loop => 145.29
    * a = 0.7
        * epsilon = 0.9 * (0.8)^loop => 49.27
    * a = 0.8
        * epsilon = 0.9 * (0.8)^loop => 96.61
    * a = 0.9
        * epsilon = 0.9 * (0.8)^loop => 160.8667968042252
    * a = 1 - 0.01*loop
        * epsilon = 0.9 * (0.8)^loop => 101.106
    * a = a*0.99, a = 1
        * epsilon = 0.9 * (0.8)^loop => 146.84
    
* action space [0, 0.2, 0.4, ..., 1.0]: epsilon = 0.9 * (0.99)^loop => 125.54

In [ ]:
class Q_Agent():
    
    def __init__(self, environment):
        
        #Hyperparameters
        self.env = environment
        self.epsilon = 0.1
        self.gamma = 0.8
        self.action_resolution = 0.2
        self.Q = defaultdict(lambda : 0.) # Q-function
        self.n = defaultdict(lambda : 1.) # number of visits
        self.actions = actionSpace(self.action_resolution)
        self.actionspace = range(len(self.actions)-1)
        
    
    def actionSpace(self):
        x,y = np.meshgrid(np.arange(0,1+self.action_resolution,self.action_resolution), np.arange(0,1+self.action_resolution,self.action_resolution))
        xy = np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)
        return xy.round(2).tolist()

    def train(self):
        
        Q = self.Q
        n = self.n
        actions = self.actions
        actionspace = self.actionspace

        greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
        max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])

        
        for _ in range(20): #Do not change
            
            self.env.reset()
            nextstate = self.env.state
            
            while True:
                state = nextstate

                # Epsilon-Greedy Action Selection
                if epsilon > random.random() :
                    action = random.choice(actionspace)
                else :
                    action = greedy_action(state)

                env_action = actions[action]#convert to ITN/IRS
                print('env_action', env_action)
                nextstate, reward, done, _ = self.env.evaluateAction(env_action)

                # Q-learning
                if done :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward - Q[(state,action)] )
                    break
                else :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )

        return Q


    def generate(self):
        best_policy = None
        best_reward = -float('Inf')
        
        Q_trained = self.train()
        greedy_eval = lambda s : max(actionspace, key=lambda a : Q_trained[(s,a)])
        
        best_policy = {state: list(actions[greedy_eval(state-1)]) for state in range(1,6)}
        best_reward = self.env.evaluatePolicy(best_policy)
        
        print(best_policy, best_reward)
        
        return best_policy, best_reward

In [ ]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, Q_Agent, "Q_submission.csv")

In [55]:
from netsapi.challenge import *

class CustomAgent:
    def __init__(self, environment):
        self.environment = environment

    def generate(self):
        best_policy = None
        best_reward = -float('Inf')
        candidates = []
        try:
            # Agents should make use of 20 episodes in each training run, if making sequential decisions
            for i in range(20):
                self.environment.reset()
                policy = {}
                for j in range(5): #episode length
                    x = 0
                    y = 0
                    ran = random.random()
                    if ran > 0.7:
                        x = 1
                    elif ran < 0.7 and ran > 0.3:
                        x = 0.5
                        y = 0.5
                    else:
                        y = 1
                    policy[str(j+1)]=[x, y]
                candidates.append(policy)
                
            rewards = self.environment.evaluatePolicy(candidates)
            best_policy = candidates[np.argmax(rewards)]
            best_reward = rewards[np.argmax(rewards)]
        
        except (KeyboardInterrupt, SystemExit):
            print(exc_info())
            
        return best_policy, best_reward

In [56]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, CustomAgent, "example.csv")

105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
105  Evaluations Remaining
399.60788879132554


* getting [0.5, 0.5]
    * 20%: all with [0.5 0.5] are worst 336, 357
    * 40%: 336, 412, 423